In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.simplefilter('ignore')


In [ ]:
data = pd.read_excel('../input/daftar-harga-rumah/DATA RUMAH.xlsx')

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
data.isnull().values.any()

In [ ]:

data = data.drop(['NO','NAMA RUMAH'],axis=1)

In [ ]:
data.head()

In [ ]:
sns.pairplot(data)

In [ ]:
plt.figure(figsize=(10,6))
sns.lineplot(x='LT',y='HARGA',data=data,ci=False)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
data.head()

In [ ]:
X = data.drop('HARGA',axis=1)
y = data['HARGA']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [ ]:
norm_X_train = X_train

In [ ]:
#norm_X_train = scaler.fit_transform(X_train)

In [ ]:
norm_X_test = X_test

In [ ]:
#norm_X_test = scaler.fit_transform(X_test)

In [ ]:
#norm_X_train = pd.DataFrame(data=norm_X_train,columns=X_train.columns)

In [ ]:
#norm_X_test = pd.DataFrame(data=norm_X_test,columns=X_test.columns)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers

In [ ]:
print(len(norm_X_train.keys()))

In [ ]:
X.head()

In [ ]:
y_test.head()

In [ ]:
def build_model():
    model = keras.Sequential([
        layers.Dense(60,activation=tf.nn.relu, input_shape=[len(norm_X_train.keys())]),
        layers.Dropout(0.5),
        layers.Dense(60, activation=tf.nn.relu,kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01)),
        layers.Dropout(0.5),
        layers.Dense(60, activation=tf.nn.relu,kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01)),
        layers.Dense(1)
          ])

    optimizer = tf.keras.optimizers.RMSprop(0.001)
    
    model.compile(loss='mean_squared_error',
                optimizer=optimizer,
                metrics=['mean_absolute_error', 'mean_squared_error'])
    return model

In [ ]:
model = build_model()

In [ ]:
model.summary()

In [ ]:
EPOCHS = 1500


In [ ]:
def plot_history(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
  
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Abs Error')
    plt.plot(hist['epoch'], hist['mean_absolute_error'],
           label='Train Error')
    plt.plot(hist['epoch'], hist['val_mean_absolute_error'],
           label = 'Val Error')
    
    plt.legend()
  
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Square Error')
    plt.plot(hist['epoch'], hist['mean_squared_error'],
           label='Train Error')
    plt.plot(hist['epoch'], hist['val_mean_squared_error'],
           label = 'Val Error')
    
    plt.legend()
    plt.show()

#plot_history(history)

In [ ]:
#hist = pd.DataFrame(history.history)
#hist['epoch'] = history.epoch
#hist.tail()

In [ ]:
model = build_model()

# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(norm_X_train, y_train, epochs=EPOCHS,batch_size=100,
                    validation_split = 0.2, verbose=0, callbacks=[early_stop])


In [ ]:
model.save('model_rev1.h5')

In [ ]:
plot_history(history)

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
norm_X_train.count()

In [ ]:
norm_X_test.head()

In [ ]:
print(model.predict(pd.DataFrame([[0,0,0,0,0],])))

In [ ]:
loss, mae, mse = model.evaluate(norm_X_train, y_train, verbose=0)

print("Training set Mean Abs Error: {:5.2f}".format(mae))

In [ ]:
data['HARGA'].mean()

In [ ]:
test_predictions = model.predict(norm_X_test).flatten()
plt.scatter(y_test, test_predictions)
print(test_predictions[1])
plt.xlabel('True prices')
plt.ylabel('Predicted prices')

In [ ]:
test_predictions.mean()

In [ ]:
y_test.mean()

In [ ]:
test_predictions.mean()

In [ ]:
loss, mae, mse = model.evaluate(norm_X_test, y_test, verbose=0)

print("Testing set Mean Abs Error: {:5.2f}".format(mae))